## RVAS on genes

In [1]:
import hail as hl
from bokeh.io import output_notebook, show
import numpy as np
import pandas as pd
import os

os.environ['JAVA_HOME']='/usr/lib/jvm/java-1.8.0-openjdk-amd64'
os.environ['PATH'] = os.environ['PATH'] + f':{os.environ["JAVA_HOME"]}/bin'
os.environ['JAVA_OPTS']='-Xmx150g'

In [2]:
hl.init(spark_conf={"spark.driver.memory": '100g'})
output_notebook()

2022-01-28 12:15:33 WARN  Utils:66 - Your hostname, bioinf resolves to a loopback address: 127.0.1.1; using 217.148.215.30 instead (on interface bond0)
2022-01-28 12:15:33 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2022-01-28 12:15:33 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-01-28 12:15:34 WARN  Hail:43 - This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.
2022-01-28 12:15:35 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-01-28 12:15:35 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2022-01-28 12:15:35 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


Running on Apache Spark version 2.4.1
SparkUI available at http://217.148.215.30:4043
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.63-cb767a7507c8
LOGGING: writing to /media/array/covid_proj/cvas_and_rvas/hail-20220128-1215-0.2.63-cb767a7507c8.log


Loading BokehJS ...

In [3]:
VCF = '/media/array/covid_proj/filtered_vcf/vep_dp_MergedVariants.renamed.vcf'
PHENO = '/media/array/covid_proj/feature_processing/data/pheno_for_analysis_imputed_filtered_INT_OCT.tsv'
SAVE_TO = './data_for_rvas/LAST_data_for_rvas_last.tsv'
PHENO = '/media/array/covid_proj/feature_processing/data/pheno_filtered_INT_sd3.tsv'
SAVE_TO = './rvas_dataset/LAST_data_for_rvas_last.tsv'
DRAW = False

In [4]:
numerical_cols = [
    "BACT_sum",
    "CT_features",
    "CT_init",
    "CT_out",
    "CT_start",
    "com_index",
    "creat",
    "creat_1",
    "creat_dynam",
    "creat_max_min",
    "crp",
    "crp_1",
    "crp_dynam",
    "crp_max_min",
    "ct1_init",
    "ct1_out",
    "ct1_start",
    "ct2_init",
    "ct2_out",
    "ct2_start",
    "days_at_hosp",
    "days_to_hosp",
    "l",
    "l_1",
    "l_3",
    "l_dynam",
    "l_max_min",
    "leuk",
    "leuk_1",
    "leuk_3",
    "leuk_7",
    "leuk_dynam",
    "leuk_max_min",
    "neut",
    "neut_1",
    "neut_3",
    "neut_dynam",
    "neut_max_min",
    "pc1",
    "pc2"
]

id_cols = ["man_id", "launch_id"]
categorical_cols = [
    "age",
    "current_situation",
    "death",
    "news",
    "severity",
    "news_dynam",
    "news_init",
    "news_max_min",
    "news_out",
    "news_start",
    "resp_dis_init",
    "resp_dis_out",
    "resp_dis_start",
    "sex",
    "storm",
    "ten_immortality",
    "week_to_hosp",
]
dont_check_cols = ["treatment"]
all_cols = id_cols + categorical_cols + numerical_cols + dont_check_cols
print(len(all_cols))

60


## Import data

In [5]:
mt = hl.import_vcf(VCF, reference_genome='GRCh38')
mt = mt.annotate_cols(genome_id=hl.int(mt.s))
mt = mt.key_cols_by("genome_id")
mt = mt.drop("s")

sa = hl.import_table(PHENO,
                     key='launch_blood_id',
                     impute=True,
                    missing='')
for col in categorical_cols:
    exec(f'sa = sa.annotate({col} = hl.int(sa["{col}"]))')
for col in id_cols:
    exec(f'sa = sa.annotate({col} = hl.int(sa["{col}"]))')
for col in numerical_cols:
    exec(f'sa = sa.annotate({col} = hl.float(sa["{col}"]))')
# sa.describe()

a = set(sa.launch_blood_id.collect())
b = set(mt.genome_id.collect())
print(len(a), len(b), len(a&b))

mt = mt.annotate_cols(pheno = sa[mt.genome_id])
mt.describe()

2022-01-28 12:15:41 Hail: INFO: Reading table to impute column types
2022-01-28 12:15:45 Hail: INFO: Loading 61 fields. Counts by type:
  float64: 56
  int32: 4
  str: 1
[Stage 2:====================================================>  (124 + 6) / 130]2022-01-28 12:15:54 Hail: INFO: Coerced sorted dataset


840 840 840
----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'genome_id': int32
    'pheno': struct {
        launch_id: int32, 
        man_id: int32, 
        storm: int32, 
        crp: float64, 
        leuk: float64, 
        neut: float64, 
        l: float64, 
        creat: float64, 
        crp_dynam: float64, 
        leuk_dynam: float64, 
        neut_dynam: float64, 
        l_dynam: float64, 
        creat_dynam: float64, 
        news_dynam: int32, 
        treatment: str, 
        com_index: float64, 
        ten_immortality: int32, 
        resp_dis_init: int32, 
        news_init: int32, 
        resp_dis_start: int32, 
        news_start: int32, 
        resp_dis_out: int32, 
        CT_init: float64, 
        ct1_init: float64, 
        ct2_init: float64, 
        CT_start: float64, 
        ct1_start: float64, 
        ct2_start: float64, 
        CT_out: float64, 
        ct1_out: float64, 

In [6]:
mt = mt.filter_rows(hl.len(mt.filters) == 0, keep=True)

### FILTERING

Filter all variants: MAF < 0.05

Other filtrations are the same, as during CVAS

In [7]:
mt = hl.sample_qc(mt)
mt.sample_qc.describe()
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.15)
mt = mt.filter_cols(mt.sample_qc.call_rate <= 0.3)
if DRAW:
    p = hl.plot.histogram(mt.sample_qc.call_rate)
    show(p)

--------------------------------------------------------
Type:
        struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transversion: int64, 
        n_star: int64, 
        r_ti_tv: float64, 
        r_het_hom_var: float64, 
        r_insertion_deletion: float64
    }
--------------------------------------------------------
Source:
Index:
    ['column']
---------------------

In [8]:
mt = hl.variant_qc(mt)
mt = mt.annotate_rows(MAF=hl.min(mt.variant_qc.AF))
mt = mt.filter_rows(mt.MAF < 0.05)
mt = mt.filter_rows(mt.variant_qc.call_rate >= 0.95)

In [9]:
if DRAW:
    p = hl.plot.histogram(mt.variant_qc.p_value_hwe)
    show(p)

In [10]:
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 0.001)
if DRAW:
    p = hl.plot.histogram(mt.variant_qc.p_value_hwe)
    show(p)

### Sample QC

In [11]:
mt = hl.sample_qc(mt)
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.95)

In [12]:
if DRAW:
    p = hl.plot.scatter(x=mt.sample_qc.r_het_hom_var, y=mt.sample_qc.call_rate)
    show(p)

In [13]:
if DRAW:
    p = hl.plot.histogram(mt.sample_qc.r_insertion_deletion)
    show(p)

In [14]:
if DRAW:
    p = hl.plot.histogram(mt.sample_qc.r_ti_tv)
    show(p)

In [15]:
if DRAW:
    p = hl.plot.scatter(x=mt.sample_qc.r_het_hom_var, y=mt.sample_qc.call_rate)
    show(p)

In [16]:
if DRAW:
    p = hl.plot.scatter(x=mt.sample_qc.r_het_hom_var, y=mt.sample_qc.call_rate)
    show(p)

In [17]:
mt.aggregate_entries(hl.agg.fraction(hl.is_defined(mt.GT)))

[Stage 3:=================================================>    (120 + 10) / 130]2022-01-28 12:16:03 Hail: INFO: Coerced sorted dataset
[Stage 6:=====================================================> (126 + 4) / 130]2022-01-28 12:16:19 Hail: INFO: Coerced sorted dataset
[Stage 9:====================================================>  (124 + 6) / 130]2022-01-28 12:16:37 Hail: INFO: Coerced sorted dataset
[Stage 10:====================================================> (127 + 3) / 130]

0.9960309204722916

In [18]:
mt.count()

[Stage 11:===================================================>  (124 + 6) / 130]2022-01-28 12:16:54 Hail: INFO: Coerced sorted dataset
[Stage 14:================================================>    (118 + 12) / 130]2022-01-28 12:17:11 Hail: INFO: Coerced sorted dataset
[Stage 17:=====================================================>(128 + 2) / 130]2022-01-28 12:17:26 Hail: INFO: Coerced sorted dataset
[Stage 18:====================================================> (127 + 3) / 130]

(90502, 768)

## Aggregate by gene

Here we use annotations of Ensembl Variant Effect Predictor (VEP).

- get the desired annotation field
- need to split the annotation on '/\|' and take the annotation only for the desired transcript (we use consequence and canonical fields)

    * Consequences: take all HIGH (https://m.ensembl.org/info/genome/variation/prediction/predicted_data.html):
        * splice_acceptor_variant	
        * splice_donor_variant	
        * stop_gained	
        * frameshift_variant	
        * stop_lost	
        * start_lost	
        * transcript_amplification
    * Canonical - YES

In [19]:
# columns with effects of snps
consequence = 1
canonical = -4

In [20]:
mt = mt.annotate_rows(kept_transcripts =
                     mt.info.CSQ.map(
                         lambda x: x.split('\|')
                         ).filter(
                         lambda tx: (
                                     (tx[consequence].contains('splice_acceptor_variant')) |
                                     (tx[consequence].contains('splice_donor_variant')) |
                                     (tx[consequence].contains('stop_gained')) |
                                     (tx[consequence].contains('frameshift_variant')) |
                                     (tx[consequence].contains('stop_lost')) |
                                     (tx[consequence].contains('start_lost')) |
                                     (tx[consequence].contains('transcript_amplification'))
                                    )
                         ).filter(
                         lambda tx: (tx[canonical].contains('YES'))
                         )
                    )

#                                      tx[consequence].contains('missense_variant')) |
#                                     (tx[consequence].contains('transcript_ablation')) |
#                                     (

# mt.info.CSQ.map(lambda x: x.split('\|')).filter(lambda tx:
# ('missense_variant' in tx[consequence]) &
# (tx[canonical] == 'YES')))

In [21]:
mt.kept_transcripts.show(5)

[Stage 19:================================================>    (119 + 11) / 130]2022-01-28 12:17:43 Hail: INFO: Coerced sorted dataset
[Stage 22:=====================================================>(129 + 1) / 130]2022-01-28 12:17:57 Hail: INFO: Coerced sorted dataset


,,
locus,alleles,kept_transcripts
locus<GRCh38>,array<str>,array<array<str>>
chr1:1014013,"[""G"",""A""]",[]
chr1:1014217,"[""C"",""T""]",[]
chr1:1014406,"[""C"",""G""]",[]
chr1:1014451,"[""C"",""T""]",[]
chr1:1014501,"[""A"",""G""]",[]


__Delete variants without suitable annotation__

In [22]:
mt = mt.filter_rows(hl.len(mt.kept_transcripts) >= 1)
mt = mt.annotate_rows(kept_transcripts=mt.kept_transcripts.head())
mt.kept_transcripts.show()

/home/achangalidi/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated method head. (Replaced by first) -- Deprecated since version 0.2.58.
  
[Stage 25:===============================================>     (116 + 14) / 130]2022-01-28 12:18:04 Hail: INFO: Coerced sorted dataset
[Stage 28:===================================================>  (125 + 5) / 130]2022-01-28 12:18:19 Hail: INFO: Coerced sorted dataset
[Stage 31:=============================>                            (1 + 1) / 2]

,,
locus,alleles,kept_transcripts
locus<GRCh38>,array<str>,array<str>
chr1:1049050,"[""TGCAGCTCAGGTGGGCGGGGAGGGGACGGGGCCGGG"",""T""]","[""-"",""splice_donor_variant&coding_sequence_variant&intron_variant"",""HIGH"",""AGRN"",""ENSG00000188157"",""Transcript"",""ENST00000379370"",""protein_coding"",""24/36"",""24/35"","""","""",""4343-?"",""4290-?"",""1430-?"","""","""",""rs1156907002"","""",""1"","""",""HGNC"",""HGNC:329"",""YES"","""","""",""""]"
chr1:2029778,"[""C"",""T""]","[""T"",""stop_gained"",""HIGH"",""GABRD"",""ENSG00000187730"",""Transcript"",""ENST00000638771"",""protein_coding"",""8/8"","""","""","""",""1120"",""1075"",""359"",""R/*"",""Cga/Tga"",""rs541562450"","""",""1"","""",""HGNC"",""HGNC:4084"",""YES"","""","""",""""]"
chr1:5867897,"[""C"",""A""]","[""A"",""splice_acceptor_variant"",""HIGH"",""NPHP4"",""ENSG00000131697"",""Transcript"",""ENST00000378156"",""protein_coding"","""",""23/29"","""","""","""","""","""","""","""",""CS139051&COSV100977135"","""",""-1"","""",""HGNC"",""HGNC:19104"",""YES"","""",""0&1"",""1&1""]"
chr1:5873294,"[""CA"",""C""]","[""-"",""frameshift_variant"",""HIGH"",""NPHP4"",""ENSG00000131697"",""Transcript"",""ENST00000378156"",""protein_coding"",""23/30"","""","""","""",""3492"",""3272"",""1091"",""V/X"",""gTg/gg"",""rs1278089386&CD022287"","""",""-1"","""",""HGNC"",""HGNC:19104"",""YES"",""pathogenic"","""",""1&1""]"
chr1:6445756,"[""A"",""ATACCCCCCCCCCCC""]","[""TACCCCCCCCCCCC"",""frameshift_variant"",""HIGH"",""ESPN"",""ENSG00000187017"",""Transcript"",""ENST00000636330"",""protein_coding"",""7/11"","""","""","""",""1453-1454"",""1285-1286"",""429"",""T/IPPPPX"",""aca/aTACCCCCCCCCCCCca"","""","""",""1"","""",""HGNC"",""HGNC:13281"",""YES"","""","""",""""]"
chr1:6445763,"[""CACCA"",""C""]","[""-"",""frameshift_variant"",""HIGH"",""ESPN"",""ENSG00000187017"",""Transcript"",""ENST00000636330"",""protein_coding"",""7/11"","""","""","""",""1461-1464"",""1293-1296"",""431-432"",""PP/X"",""ccACCA/cc"",""rs760964146"","""",""1"","""",""HGNC"",""HGNC:13281"",""YES"","""","""",""""]"
chr1:6474166,"[""T"",""C""]","[""C"",""splice_acceptor_variant"",""HIGH"",""PLEKHG5"",""ENSG00000171680"",""Transcript"",""ENST00000673471"",""protein_coding"","""",""7/21"","""","""","""","""","""","""","""",""rs144750655"","""",""-1"","""",""HGNC"",""HGNC:29105"",""YES"",""uncertain_significance&likely_pathogenic"","""",""1""]"
chr1:10475000,"[""C"",""T""]","[""T"",""stop_gained&splice_region_variant"",""HIGH"",""PEX14"",""ENSG00000142655"",""Transcript"",""ENST00000356607"",""protein_coding"",""1/9"","""","""","""",""51"",""34"",""12"",""Q/*"",""Cag/Tag"","""","""",""1"","""",""HGNC"",""HGNC:8856"",""YES"","""","""",""""]"


__Take gene_symbol and other fields, that we will need for filtration__

In [23]:
mt = mt.annotate_rows(kept_transcripts = mt.kept_transcripts[3])

In [24]:
mt.kept_transcripts.show(5)

[Stage 32:=====================================================>(129 + 1) / 130]2022-01-28 12:18:31 Hail: INFO: Coerced sorted dataset
[Stage 35:============================================>        (110 + 20) / 130]2022-01-28 12:18:45 Hail: INFO: Coerced sorted dataset
[Stage 37:>                                                         (0 + 1) / 1]

,,
locus,alleles,kept_transcripts
locus<GRCh38>,array<str>,str
chr1:1049050,"[""TGCAGCTCAGGTGGGCGGGGAGGGGACGGGGCCGGG"",""T""]","""AGRN"""
chr1:2029778,"[""C"",""T""]","""GABRD"""
chr1:5867897,"[""C"",""A""]","""NPHP4"""
chr1:5873294,"[""CA"",""C""]","""NPHP4"""
chr1:6445756,"[""A"",""ATACCCCCCCCCCCC""]","""ESPN"""


In [25]:
mt.count()

[Stage 38:==================================================>   (121 + 9) / 130]2022-01-28 12:18:53 Hail: INFO: Coerced sorted dataset
[Stage 41:====================================================> (127 + 3) / 130]2022-01-28 12:19:08 Hail: INFO: Coerced sorted dataset
[Stage 44:=====================================================>(128 + 2) / 130]2022-01-28 12:19:25 Hail: INFO: Coerced sorted dataset
[Stage 45:=====================================================>(129 + 1) / 130]

(1884, 768)

__Group by genes for each patient and pull out this info__

In [26]:
mt_new = mt.group_rows_by(mt.kept_transcripts) \
    .aggregate(n_het=hl.agg.count_where(mt.GT.is_het()), 
               n_hom_ref=hl.agg.count_where(mt.GT.is_hom_ref()), 
               n_hom_var=hl.agg.count_where(mt.GT.is_hom_var())
                   )

__To pandas__

In [27]:
tbl = mt_new.entries()
data_d = tbl.to_pandas()
print(data_d.shape)
data_d.head()

2022-01-28 12:19:39 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'
[Stage 47:================================================>    (118 + 12) / 130]2022-01-28 12:19:44 Hail: INFO: Coerced sorted dataset
[Stage 51:=====================================================>(128 + 2) / 130]2022-01-28 12:19:57 Hail: INFO: Coerced sorted dataset
[Stage 55:===================================================>  (125 + 5) / 130]2022-01-28 12:20:21 Hail: INFO: Coerced sorted dataset
[Stage 57:=============================================>       (111 + 19) / 130]2022-01-28 12:20:37 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 60:==============================================>      (113 + 17) / 130]

(860928, 91)


,kept_transcripts,genome_id,pheno.launch_id,pheno.man_id,pheno.storm,pheno.crp,pheno.leuk,pheno.neut,pheno.l,pheno.creat,...,sample_qc.n_deletion,sample_qc.n_transition,sample_qc.n_transversion,sample_qc.n_star,sample_qc.r_ti_tv,sample_qc.r_het_hom_var,sample_qc.r_insertion_deletion,n_het,n_hom_ref,n_hom_var
0,A2M,637,63,626630,0.0,-0.697821,-2.168169,-1.909299,-1.288741,-0.298489,...,65,1490,531,0,2.806026,0.770725,0.815385,0,1,0
1,A2M,639,63,620762,0.0,1.101656,0.307014,0.426411,-0.617996,-0.769107,...,66,1461,543,0,2.690608,0.776762,0.863636,0,1,0
2,A2M,881,88,621011,0.0,0.350675,-0.239527,-0.022812,-0.169224,-2.013296,...,56,1380,494,0,2.793522,0.746871,0.803571,0,1,0
3,A2M,882,88,627130,0.0,-0.179248,0.150065,0.163654,-0.194830,-1.988692,...,61,1512,527,0,2.869070,0.954608,0.786885,0,1,0
4,A2M,883,88,627508,0.0,-1.236441,-1.380005,-1.029558,-0.262704,-1.302998,...,58,1476,557,0,2.649910,0.990196,0.758621,0,1,0


In [28]:
from scipy.stats import fisher_exact
import seaborn as sns
import matplotlib.pyplot as plt

One person can have several mutations, so we have to transform information into 2 columns:
* individual has mutation (`if n_het > 0 or n_hom_var > 0`)
* doesn't have (`else`)

In [29]:
data_d['is_alter'] = data_d.loc[:,['n_het', 'n_hom_var']].sum(axis = 1).apply(lambda x: int(x > 0))
data_d['ref'] = data_d.loc[:,['is_alter']].apply(lambda x: 1-x)
data_d.head()

,kept_transcripts,genome_id,pheno.launch_id,pheno.man_id,pheno.storm,pheno.crp,pheno.leuk,pheno.neut,pheno.l,pheno.creat,...,sample_qc.n_transversion,sample_qc.n_star,sample_qc.r_ti_tv,sample_qc.r_het_hom_var,sample_qc.r_insertion_deletion,n_het,n_hom_ref,n_hom_var,is_alter,ref
0,A2M,637,63,626630,0.0,-0.697821,-2.168169,-1.909299,-1.288741,-0.298489,...,531,0,2.806026,0.770725,0.815385,0,1,0,0,1
1,A2M,639,63,620762,0.0,1.101656,0.307014,0.426411,-0.617996,-0.769107,...,543,0,2.690608,0.776762,0.863636,0,1,0,0,1
2,A2M,881,88,621011,0.0,0.350675,-0.239527,-0.022812,-0.169224,-2.013296,...,494,0,2.793522,0.746871,0.803571,0,1,0,0,1
3,A2M,882,88,627130,0.0,-0.179248,0.150065,0.163654,-0.194830,-1.988692,...,527,0,2.869070,0.954608,0.786885,0,1,0,0,1
4,A2M,883,88,627508,0.0,-1.236441,-1.380005,-1.029558,-0.262704,-1.302998,...,557,0,2.649910,0.990196,0.758621,0,1,0,0,1


### Save data

In [30]:
data_d.to_csv(SAVE_TO, sep='\t', index=False)

## 